## Libraries

In [1]:
from skimage import transform
import glob
import pandas
import os
import scipy.io
import cv2
import matplotlib.pyplot as plt 
from shutil import copyfile
%matplotlib inline

## Label list

In [2]:
path="../../data/Clothing Co-Parsing"

In [3]:
label_list=scipy.io.loadmat(f'{path}/label_list.mat')

In [4]:
class_names=( { w:v for v,w in enumerate(map(lambda x: x[0],label_list["label_list"][0]))})

## Select clothes

In [5]:
clothes_interest=['blazer','blouse','bodysuit','cardigan','coat','hoodie','jacket','jeans','jumper','leggings','pants',
                  'romper','shirt','shorts','suit','sweater','sweatshirt','t-shirt','vest']


clothes_sup=list(map(lambda x: (class_names[x],"sup",x),["blazer", "blouse", "coat","jacket","shirt","sweater","sweatshirt",
                                     "vest","hoodie","cardigan","bodysuit","jumper",
                                     "suit","romper","t-shirt"]))
clothes_inf=[(class_names[clth],"inf",clth) for clth in clothes_interest if not clth in ["blazer", "blouse", "coat",
                                                                                         "jacket","shirt","sweater","sweatshirt","vest","hoodie",
                                                                                         "cardigan","bodysuit","jumper","suit","romper","t-shirt"]]

ids_sup=[x[0] for x in clothes_sup]
ids_inf=[x[0] for x in clothes_inf]


In [6]:
len(clothes_interest)-(len(clothes_sup)+len(clothes_inf))

0

## Generate arrays

In [8]:
list_segs=[]
list_imgs=[]
for x in glob.glob(f"{path}/all_imgs/*.jpg"):
    seg_array=scipy.io.loadmat(x.replace("jpg","mat"))["groundtruth"]
    seg_array[~np.isin(seg_array, np.array(ids_sup+ids_inf))]=0
    seg_array=(seg_array.reshape((*seg_array.shape,1)))
    seg_array=np.concatenate([seg_array==0, np.isin(seg_array,ids_sup),np.isin(seg_array,ids_inf)],axis=2)
    list_segs.append(transform.resize(seg_array,(512,832)).round().astype(np.uint8))
    list_imgs.append(transform.resize(plt.imread(x),(512,832)).round().astype(np.uint8))    

In [9]:
list_imgs=np.array(list_imgs,dtype=np.uint8)

In [10]:
list_segs=np.array(list_segs,dtype=np.uint8)

In [11]:
np.savez("../../data/segs_unet",(list_segs))
np.savez("../../data/imgs_unet",(list_imgs))